# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('../data/sp_500_stocks.csv')
from secrets_api import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 67647792,
 'calculationPrice': 'close',
 'change': -6.46,
 'changePercent': -0.0394,
 'close': 169.71,
 'closeSource': 'coaifilf',
 'closeTime': 1683717824109,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 166.43,
 'delayedPriceTime': 1667597569482,
 'extendedChange': -1.2,
 'extendedChangePercent': -0.00731,
 'extendedPrice': 166.78,
 'extendedPriceTime': 1732022001249,
 'high': 174.97,
 'highSource': 'tea5erdpu   myniecd1eil',
 'highTime': 1738416841265,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 164.69,
 'iexCloseTime': 1730290421786,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 172.51,
 'iexOpenTime': 1722440124889,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1674588776332,
 'latestPrice': 166.33,
 'latestSource': 'Close',
 'latestTime': 'August 26, 2022',
 'latestUpdate': 1739136874269,
 'latestVolume': 79527102,
 'low': 1

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

28.15

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    api_batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_batch_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            df = df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ], index=my_columns),
            ignore_index=True)

df

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/1969563

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/196956359.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/1969563

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,130.33,32.54,N/A
1,AAL,14.38,-4.79,N/A
2,AAP,180.50,20.8,N/A
3,AAPL,171.64,28.12,N/A
4,ABBV,141.16,19.4,N/A
...,...,...,...,...
496,YUM,114.23,22.92,N/A
497,ZBH,110.89,101.21,N/A
498,ZBRA,308.40,34.15,N/A
499,ZION,57.66,6.41,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
df.sort_values('Price-to-Earnings Ratio', inplace=True)
df = df[df['Price-to-Earnings Ratio'] > 0]
df = df[:50]
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.27,3.02,N/A
1,AIG,55.00,3.58,N/A
2,NUE,142.33,4.37,N/A
3,PHM,42.33,4.62,N/A
4,KSS,30.75,4.63,N/A
5,PVH,66.83,4.74,N/A
6,PFG,77.55,4.78,N/A
7,APA,41.10,4.82,N/A
8,MGM,35.23,4.83,N/A
9,DHI,73.66,4.86,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
portfolio_size

'1000000'

In [11]:
position_size = float(portfolio_size)/len(df.index)
for row in df.index:
    df.loc[row, 'Number of Shares to Buy'] = position_size/df.loc[row,'Price']

df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.27,3.02,473.148805
1,AIG,55.00,3.58,363.636364
2,NUE,142.33,4.37,140.518513
3,PHM,42.33,4.62,472.478148
4,KSS,30.75,4.63,650.406504
5,PVH,66.83,4.74,299.266796
6,PFG,77.55,4.78,257.89813
7,APA,41.10,4.82,486.618005
8,MGM,35.23,4.83,567.697985
9,DHI,73.66,4.86,271.517784


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'

batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gp = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = ev/gp


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [13]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earn Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        #print(data[symbol]['quote'])
        if symbol in data.keys():
            ev = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gp = data[symbol]['advanced-stats']['grossProfit']

            try:
                ev_to_ebitda = ev/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN
            
            try:
                ev_to_gp = ev/gp
            except TypeError:
                ev_to_gp = np.NaN
                
            rv_df = rv_df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
            ], index=rv_columns),
            ignore_index=True)

rv_df

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52577/4062581509.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b4

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,133.10,N/A,32.8,N/A,7.67,N/A,6.15,N/A,22.657076,N/A,11.724385,N/A,N/A
1,AAL,14.15,N/A,-4.82,N/A,-1.08,N/A,0.2261,N/A,112.790944,N/A,0.822803,N/A,N/A
2,AAP,178.97,N/A,20.91,N/A,3.81,N/A,0.9831,N/A,11.363220,N/A,2.358310,N/A,N/A
3,AAPL,168.20,N/A,27.07,N/A,45.45,N/A,7.02,N/A,20.233604,N/A,15.590780,N/A,N/A
4,ABBV,142.70,N/A,19.96,N/A,17.07,N/A,4.2,N/A,10.620025,N/A,7.544498,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,117.07,N/A,22.67,N/A,-3.84,N/A,4.8,N/A,20.102117,N/A,8.704659,N/A,N/A
497,ZBH,109.64,N/A,100.9,N/A,1.89,N/A,3.06,N/A,17.734516,N/A,5.394838,N/A,N/A
498,ZBRA,310.25,N/A,33.71,N/A,6.47,N/A,2.81,N/A,17.018893,N/A,7.025887,N/A,N/A
499,ZION,57.05,N/A,6.4,N/A,1.66,N/A,2.47,N/A,5.117134,N/A,2.438406,N/A,N/A


In [14]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,133.10,N/A,32.8,N/A,7.67,N/A,6.15,N/A,22.657076,N/A,11.724385,N/A,N/A
1,AAL,14.15,N/A,-4.82,N/A,-1.08,N/A,0.2261,N/A,112.790944,N/A,0.822803,N/A,N/A
2,AAP,178.97,N/A,20.91,N/A,3.81,N/A,0.9831,N/A,11.363220,N/A,2.358310,N/A,N/A
3,AAPL,168.20,N/A,27.07,N/A,45.45,N/A,7.02,N/A,20.233604,N/A,15.590780,N/A,N/A
4,ABBV,142.70,N/A,19.96,N/A,17.07,N/A,4.2,N/A,10.620025,N/A,7.544498,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,117.07,N/A,22.67,N/A,-3.84,N/A,4.8,N/A,20.102117,N/A,8.704659,N/A,N/A
497,ZBH,109.64,N/A,100.9,N/A,1.89,N/A,3.06,N/A,17.734516,N/A,5.394838,N/A,N/A
498,ZBRA,310.25,N/A,33.71,N/A,6.47,N/A,2.81,N/A,17.018893,N/A,7.025887,N/A,N/A
499,ZION,57.05,N/A,6.4,N/A,1.66,N/A,2.47,N/A,5.117134,N/A,2.438406,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [15]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,185.700,N/A,59.67,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,287.520,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,290.520,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,99.360,N/A,50.15,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.84,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.690,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,50.810,N/A,14.58,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.730,N/A,33.6,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,32.320,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,105.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [16]:
for col in ['Price-to-Earn Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_df[col].fillna(rv_df[col].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [18]:
metrics = {
    'Price-to-Earn Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric]) / 100

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,133.10,N/A,32.80,0.744511,7.67,0.772455,6.1500,0.832335,22.657076,0.822355,11.724385,0.820359,N/A
1,AAL,14.15,N/A,-4.82,0.033932,-1.08,0.05988,0.2261,0.00998,112.790944,0.996008,0.822803,0.02994,N/A
2,AAP,178.97,N/A,20.91,0.497006,3.81,0.55489,0.9831,0.145709,11.363220,0.37525,2.358310,0.141717,N/A
3,AAPL,168.20,N/A,27.07,0.646707,45.45,0.978044,7.0200,0.856287,20.233604,0.756487,15.590780,0.928144,N/A
4,ABBV,142.70,N/A,19.96,0.469062,17.07,0.928144,4.2000,0.715569,10.620025,0.325349,7.544498,0.556886,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,117.07,N/A,22.67,0.540918,-3.84,0.055888,4.8000,0.754491,20.102117,0.748503,8.704659,0.684631,N/A
497,ZBH,109.64,N/A,100.90,0.978044,1.89,0.269461,3.0600,0.546906,17.734516,0.694611,5.394838,0.387226,N/A
498,ZBRA,310.25,N/A,33.71,0.755489,6.47,0.734531,2.8100,0.499002,17.018893,0.672655,7.025887,0.510978,N/A
499,ZION,57.05,N/A,6.40,0.08982,1.66,0.213573,2.4700,0.443114,5.117134,0.095808,2.438406,0.147705,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
from statistics import mean

for row in rv_df.index:
    value_percentile = []
    for metric in metrics.keys():
        value_percentile.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentile)

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,133.10,N/A,32.80,0.744511,7.67,0.772455,6.1500,0.832335,22.657076,0.822355,11.724385,0.820359,0.798403
1,AAL,14.15,N/A,-4.82,0.033932,-1.08,0.05988,0.2261,0.00998,112.790944,0.996008,0.822803,0.02994,0.225948
2,AAP,178.97,N/A,20.91,0.497006,3.81,0.55489,0.9831,0.145709,11.363220,0.37525,2.358310,0.141717,0.342914
3,AAPL,168.20,N/A,27.07,0.646707,45.45,0.978044,7.0200,0.856287,20.233604,0.756487,15.590780,0.928144,0.833134
4,ABBV,142.70,N/A,19.96,0.469062,17.07,0.928144,4.2000,0.715569,10.620025,0.325349,7.544498,0.556886,0.599002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,117.07,N/A,22.67,0.540918,-3.84,0.055888,4.8000,0.754491,20.102117,0.748503,8.704659,0.684631,0.556886
497,ZBH,109.64,N/A,100.90,0.978044,1.89,0.269461,3.0600,0.546906,17.734516,0.694611,5.394838,0.387226,0.57525
498,ZBRA,310.25,N/A,33.71,0.755489,6.47,0.734531,2.8100,0.499002,17.018893,0.672655,7.025887,0.510978,0.634531
499,ZION,57.05,N/A,6.40,0.08982,1.66,0.213573,2.4700,0.443114,5.117134,0.095808,2.438406,0.147705,0.198004


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_df = rv_df.sort_values('RV Score')[:50]
rv_df.reset_index(drop=True, inplace=True)
#rv_df.drop(columns=['index'], inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,30.95,N/A,4.68,0.055888,0.8640,0.081836,0.2009,0.007984,3.302997,0.031936,0.958980,0.03992,0.043513
1,AIG,55.00,N/A,3.66,0.0499,0.9054,0.08982,0.7218,0.102794,2.101906,0.015968,0.723457,0.015968,0.05489
2,PVH,66.97,N/A,4.69,0.057884,0.8456,0.077844,0.4754,0.047904,4.768577,0.07984,1.062330,0.0499,0.062675
3,GPS,9.97,N/A,-53.92,0.011976,1.4800,0.172655,0.2287,0.011976,5.250835,0.097804,0.762489,0.01996,0.062874
4,UNM,40.31,N/A,7.43,0.117764,0.8146,0.073852,0.6682,0.08982,4.198627,0.05988,0.653711,0.00998,0.070259
5,GM,40.38,N/A,7.60,0.122754,0.9225,0.095808,0.4480,0.045908,2.309790,0.017964,1.885067,0.103792,0.077246
6,SYF,33.75,N/A,4.90,0.06487,1.3200,0.145709,0.9232,0.135729,3.002606,0.01996,0.982964,0.041916,0.081637
7,CAH,70.00,N/A,-20.38,0.01996,-26.7400,0.027944,0.1057,0.001996,7.262086,0.181637,2.905102,0.191617,0.084631
8,COF,110.85,N/A,4.90,0.06487,0.7988,0.071856,1.2500,0.209581,3.300839,0.02994,1.307648,0.067864,0.088822
9,C,51.37,N/A,6.49,0.093812,0.5425,0.063872,1.1500,0.183633,3.845748,0.045908,1.186575,0.063872,0.09022


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()

Enter the value of your portfolio:1000000


In [22]:
position_size = float(portfolio_size)/len(rv_df.index)

for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = position_size/rv_df.loc[row, 'Price']

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earn Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,30.95,646.203554,4.68,0.055888,0.8640,0.081836,0.2009,0.007984,3.302997,0.031936,0.958980,0.03992,0.043513
1,AIG,55.00,363.636364,3.66,0.0499,0.9054,0.08982,0.7218,0.102794,2.101906,0.015968,0.723457,0.015968,0.05489
2,PVH,66.97,298.641183,4.69,0.057884,0.8456,0.077844,0.4754,0.047904,4.768577,0.07984,1.062330,0.0499,0.062675
3,GPS,9.97,2006.018054,-53.92,0.011976,1.4800,0.172655,0.2287,0.011976,5.250835,0.097804,0.762489,0.01996,0.062874
4,UNM,40.31,496.1548,7.43,0.117764,0.8146,0.073852,0.6682,0.08982,4.198627,0.05988,0.653711,0.00998,0.070259
5,GM,40.38,495.2947,7.60,0.122754,0.9225,0.095808,0.4480,0.045908,2.309790,0.017964,1.885067,0.103792,0.077246
6,SYF,33.75,592.592593,4.90,0.06487,1.3200,0.145709,0.9232,0.135729,3.002606,0.01996,0.982964,0.041916,0.081637
7,CAH,70.00,285.714286,-20.38,0.01996,-26.7400,0.027944,0.1057,0.001996,7.262086,0.181637,2.905102,0.191617,0.084631
8,COF,110.85,180.423996,4.90,0.06487,0.7988,0.071856,1.2500,0.209581,3.300839,0.02994,1.307648,0.067864,0.088822
9,C,51.37,389.332295,6.49,0.093812,0.5425,0.063872,1.1500,0.183633,3.845748,0.045908,1.186575,0.063872,0.09022


In [23]:
# Checksum that we're spending the right amount of money!
amount = []
for row in rv_df.index:
    amount.append(rv_df.loc[row, 'Number of Shares to Buy'] * rv_df.loc[row, 'Price'])

sum(amount)

1000000.0

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [24]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {
                    'A':['Ticker', string_template],
                    'B':['Price', dollar_template],
                    'C':['Number of Shares to Buy', integer_template],
                    'D':['Price-to-Earn Ratio', float_template],
                    'E':['PE Percentile', percent_template],
                    'F':['Price-to-Book Ratio', float_template],
                    'G':['PB Percentile', percent_template],
                    'H':['Price-to-Sales Ratio', float_template],
                    'I':['PS Percentile', percent_template],
                    'J':['EV/EBITDA', float_template],
                    'K':['EV/EBITDA Percentile', percent_template],
                    'L':['EV/GP', percent_template],
                    'M':['EV/GP Percentile', percent_template],
                    'N':['RV Score', float_template]
                }

for col in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{col}:{col}', 25, column_formats[col][1])
    writer.sheets['Value Strategy'].write(f'{col}1', column_formats[col][0], column_formats[col][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [27]:
writer.save()